## Zadanie domowe - Algorytm Canny'ego

Celem zadania domowego jest wykonanie pełnej implementacji algorytmu Canny'ego.

W ramach ćwiczenia w trakcie laboratorium wyznaczono obrazy $g_{NH}$ i $g_{NL}$.
Dla przypomnienia:
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
Często krawędzie "pewne" nie są ciągłe.
Wykorzystuje się więc krawędzie "potencjalne", aby uzupełnić nieciągłości.
Procedura wygląda następująco:
1. Stwórz stos zawierający wszystkie piksele zaznaczone na obrazie $g_{NH}$.
W tym celu wykorzystaj listę współrzędnych `[row, col]`.
Do pobrania elementu z początku służy metoda `list.pop()`.
Do dodania elementu na koniec listy służy metoda `list.append(new)`.
2. Stwórz obraz, który będzie zawierał informację czy dany piksel został już odwiedzony.
3. Stwórz obraz, który zawierać będzie wynikowe krawędzie.
Jej rozmiar jest równy rozmiarowi obrazu.
4. Wykonaj pętlę, która będzie pobierać elementy z listy, dopóki ta nie będzie pusta.
W tym celu najlepiej sprawdzi się pętla `while`.
    - W każdej iteracji pobierz element ze stosu.
    - Sprawdź, czy dany element został już odwiedzony.
    - Jeśli nie został, to:
        - Oznacz go jako odwiedzony,
        - Oznacz piksel jako krawędź w wyniku,
        - Sprawdź otoczenie piksela w obrazie $g_{NL}$,
        - Dodaj do stosu współrzędne otoczenia, które zawierają krawędź.
        Można to wykonać np. pętlą po stworzonym otoczeniu.
7. Wyświetl obraz oryginalny, obraz $g_{NH}$ oraz obraz wynikowy.
8. Porównaj wynik algorytmu z wynikiem OpenCV.

Pomocnicze obrazy $g_{NH}$ i $g_{NL}$ zostały wprowadzone dla uproszczenia opisu.
Algorytm można zaimplementować w bardziej "zwarty" sposób.

Na podstawie powyższego opisu zaimplementuj pełny algorytm Canny'ego.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

if not os.path.exists("dom.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/09_Canny/dom.png --no-check-certificate

house = cv2.imread('dom.png', cv2.IMREAD_GRAYSCALE)

def img_show(image):
    plt.figure()
    plt.imshow(image, cmap="gray")
    plt.xticks([]), plt.yticks([])  # Hides the graph ticks and x / y axis
    plt.show()

In [ ]:
def nonmax(matrix, amp):
    x, y = matrix.shape
    result = np.zeros((x, y))

    for i in range(1, x-1):
        for j in range(1, y-1):
            if (matrix[i, j] == 1):
                if (amp[i, j-1] > amp[i, j] or amp[i, j+1] > amp[i, j]):
                    result[i, j] = 0
                else:
                    result[i, j] = amp[i, j]
            elif (matrix[i, j] == 2):
                if (amp[i+1, j-1] > amp[i, j] or amp[i-1, j+1] > amp[i, j]):
                    result[i, j] = 0
                else:
                    result[i, j] = amp[i, j]
            elif (matrix[i, j] == 3):
                if (amp[i+1, j] > amp[i, j] or amp[i-1, j] > amp[i, j]):
                    result[i, j] = 0
                else:
                    result[i, j] = amp[i, j]
            elif (matrix[i, j] == 4):
                if (amp[i-1, j-1] > amp[i, j] or amp[i+1, j+1] > amp[i, j]):
                    result[i, j] = 0
                else:
                    result[i, j] = amp[i, j]
    return result


def canny(image, thr_l, thr_h):
    x, y = image.shape
    img_blur = cv2.GaussianBlur(image, (5, 5), 0)
    S1 = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    S2 = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
    Gx = cv2.filter2D(img_blur, -1, S1)
    Gy = cv2.filter2D(img_blur, -1, S2)
    M = np.sqrt(Gx**2+Gy**2)
    alpha = np.arctan2(Gy, Gx)
    alpha = (alpha * 180)/np.pi
    alpha[alpha < 0] += 180
    result = np.zeros((x, y))

    for i in range(x):
        for j in range(y):
            # kąt 0
            if (0 <= alpha[i, j] < 22.5) or (157.5 <= alpha[i, j] <= 180):
                result[i, j] = 1
            # kąt 45
            elif (22.5 <= alpha[i, j] < 67.5):
                result[i, j] = 2
            # kąt 90
            elif (67.5 <= alpha[i, j] < 112.5):
                result[i, j] = 3
            # kąt 135
            elif (112.5 <= alpha[i, j] < 157.5):
                result[i, j] = 4

    result_new = nonmax(result, M)
    result_new_h = np.where(result_new > thr_h, 1, 0)
    result_new_l = np.where(np.logical_and(thr_h > result_new, result_new >= thr_l), 1, 0)
    return result_new_l, result_new_h